# New design

ttk.Frame &rarr; ListBoxFrame, ARCFrame, EncDecFrame

In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox

# Skeleton &rarr; actual code

## 1. ListBox + ARCFrame integration = ListBoxFrame

The listbox and the Add-Remove-Clear frame will be used as
one single frame.

### 1.1 ListBox

In [ ]:
class ListBox(tk.Listbox):
    """The List box.
    
    add -- Add items
    remove -- remove items
    clear -- clear items
    get -- get all items
    """
    def __init__(self, *args, master, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        self.__items = tk.Variable(master, name="__items")
        self.config(listvariable=self.__items)
        
        # x and y scrollbars with correct orientation
        xscrollbar = ttk.Scrollbar(self, orient='horizontal')
        yscrollbar = ttk.Scrollbar(self, orient='vertical')

        # pack them on the listbox
        xscrollbar.pack(side='bottom', fill='x')
        yscrollbar.pack(side='right', fill='y')

        # configure the listbox
        self.configure(
            xscrollcommand=xscrollbar.set,
            yscrollcommand=yscrollbar.set,
        )

        # set the scrollbar commands
        xscrollbar.config(command=self.xview)
        yscrollbar.config(command=self.yview)
        
        # TODO: Fix the flashing of window caused by the addition
        # of scrollbars.

    @property
    def items(self):
        return self.getvar('__items') or ()

    def get(self):
        """Get the items from the ListBox"""
        return self._items.get()

    def add(self, item):
        """Add a unique item in the ListBox"""
        if item not in self.items:
            self.insert('end', item)

    def add_many(self, *items):
        """Add several items at once."""
        for item in items:
            self.add(item)

    def clear(self):
        """Clear the ListBox"""
        self.delete(0, 'end')

    remove_all = clear

    def remove(self):
        """Remove a single selected item from the ListBox"""
        try:
            idx = self.curselection()[0]
            self.delete(idx)
        except IndexError:
            messagebox.showerror("Error", "No file selected.")

### 1.2 ARCFrame

In [ ]:
class ARCFrame(ttk.Frame):
    """Add, Remove, Clear Frame;
        * controls ListBoxFrame
    
    listbox -- the listbox to control
    on_add -- Add ctrl.
    on_remove -- rm ctrl.
    on_clear -- clearing ctrl. Aliased to 'Remove All'
    """
    def __init__(self, *args, master, listbox=None, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        self._listbox = listbox  # assume packed: EIBTI

        self._badd = ttk.Button(
            self,
            text="Add",
            command=self.on_add,
        )
        self._bremove = ttk.Button(
            self,
            text="Remove",
            command=self.on_remove,
        )
        self._bclear = ttk.Button(
            self,
            text="Clear",
            command=self.on_clear,
        )

        self._badd.grid(row=0, column=0, sticky='nsew')
        self._bremove.grid(row=0, column=1, sticky='nsew', padx=10)
        self._bclear.grid(row=0,column=2, sticky='nsew')

        self.rowconfigure(0, weight=1)
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=1)
        self.columnconfigure(2, weight=1)

    def on_add(self):
        """Open a DialogBox to select item(s) and add it to the listbox."""
        filepath = filedialog.askopenfilenames()
        if not filepath:
            return
        self._listbox.add_many(*filepath)

    def on_remove(self):
        """Remove a selected item from the listbox (Button mapped)"""
        self._listbox.remove()

    def on_clear(self):
        """Clear (or remove_all) the listbox (button mapped)"""
        self._listbox.clear()

    on_remove_all = on_clear

### * ListBoxFrame: Not to be used, but good for testing out things.

~The ListBox and the ARC-buttons are merged together in one frame.~
~They can be modified here.~

### <font color=red>The listbox accepting frames must be subclassed.</font>

In [ ]:
class ListBoxFrame(ARCFrame, ttk.Frame):
    def __init__(self, *args, master, **kwargs):
        """Integrate listbox and ARC-buttons. Customize as required."""
        self.listbox = ListBox(*args, master=master, **kwargs)
        self.listbox.pack()
        super().__init__(*args, master=master, listbox=self.listbox, **kwargs)

## 1.3 EncDecFrame: This may change!

Control the encryption and decryption.

1. Fetch the file names from ListBox,
2. Set up a **Thread-pool** or **process-pool**, (in a separate design?)
3. Yield as operate,
4. Update with colors,
5. Show stat in the end


### Problem 1:

The EncDecFrame needs to control the listbox, but unfortunately, that will
be used by the ListBoxFrame. That means, there will be no ***global***
reference to the ListBox object.

### Possible solution 1:

Since ListBox can be shared by both ARCFrame and EncDecFrame, without affecting
each other, i.e.:

                     listbox  <-- shared
                        |
              |---------|---------|
              |                   |
           ARCFrame          EncDecFrame

Subclassing it as:
              
           ARCFrame          EncDecFrame  <-- * Both take listbox value
              |                   |
              |---------|---------|
                        |
                   ControlFrame (or some better name :p)
                   
will provide a good control over both the frames.

### Possible solution 2:

Instead of subclassing, make `ControlFrame` a simple `ttk.Frame` and set up
the `ARCFrame` and `EncDecFrame` there.

> There is more than one way to do it.
>> And subclassing is not necessarily the one among them.

### But following solution 1 causes Problem 2:

The superclasses require listbox value, but `super()` only goes 1 class up.
After that, the listbox is lost, and `TypeError` is raised.

### Possible Solution:

Make `listbox` an assumed variable. The superclasses must expect that the listbox
is coming if the subclass has a `self._listbox` variable.

> This scheme is really irritating and I want to get rid of this.

### Then, what is the best solution?

#### [Solution 2 of Problem 1](#Possible-solution-2:)

In [ ]:
class EncDecFrame(ttk.Frame):
    """Encrypt / Decrypt buttons w/ a password entry.
        * Controls ListBox

    on_encrypt -- enc
    on_decrypt -- dec
    """
    def __init__(self, *args, master, listbox, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        self._listbox = listbox  # assume packed: EIBTI
        
        # TODO: make a password entry field.

        self._bencrypt = ttk.Button(
            self,
            text="Encrypt",
            command=self.on_encrypt,
        )
        self._bdecrypt = ttk.Button(
            self,
            text="Decrypt",
            command=self.on_decrypt,
        )

        # Row 0: [Frame: [Label: Password-entry]]
        pwd_frame = ttk.Frame(self)
        self._entry_pwd = ttk.Entry(pwd_frame, show="\u2022")

        ttk.Label(pwd_frame, text="Password:").grid(
            row=0,
            column=0,
            sticky="w",
            padx=(0, 20),
        )
        self._entry_pwd.grid(
            row=0,
            column=1,
            sticky="ew",
        )
        pwd_frame.grid(
            row=0,
            column=0,
            columnspan=2,
            sticky="ew",
            pady=(0, 20),
        )
        pwd_frame.columnconfigure(1, weight=1)  # let the ttk.Entry expand

        # Row 1: [Encrypt, Decrypt]
        self._bencrypt.grid(
            row=1,
            column=0,
            sticky="nsew",
        )
        self._bdecrypt.grid(
            row=1,
            column=1,
            sticky="nsew",
        )

        self.rowconfigure(1, weight=1)  # expand the buttons row-wise
        self.columnconfigure(0, weight=1)  # expand buttons columnwise
        self.columnconfigure(1, weight=1)  # expand buttons columnwise

    def on_encrypt(self):
        """Encrypt everything in the listbox."""
        print("encrypting")
        print(repr(self._listbox.items), self._entry_pwd.get())
    
    def on_decrypt(self):
        """Decrypt everything in the listbox."""
        print("decrypting")
        print(repr(self._listbox.items), self._entry_pwd.get())

## Setting up EncDecFrame + ARCFrame

## The Plan

The box must be displayed as:

    +--------------------------------------------------+
    |                                                  |
    | +----------------------------------------------+ |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | |                                              | |
    | +----------------------------------------------+ |
    |                                                  |
    | +--------------+ +-------------+ +-------------+ |
    | |     Add      | |   Remove    | |   Clear     | |
    | +--------------+ +-------------+ +-------------+ |
    |                                                  |
    | +----------------------------------------------+ |
    | |                 password                     | |
    | +----------------------------------------------+ |
    |                                                  |
    | +---------------------+   +--------------------+ |
    | |       Encrypt       |   |     Decrypt        | |
    | +---------------------+   +--------------------+ |
    |                                                  |
    +--------------------------------------------------+

By the diagram above, we have to design this such that:

1. The listbox must adapt to resize.

2. The buttons will grow **in the XY-axis** and the password field will grow **only in the X-axis**.

3. The padding must be performed by the bounding frame, because
   it is easier to maintain.

4. **The buttons and the password field will share the y-alignment with the listbox.**

Hopefully, #4 will be easier to implement as the whole `ControlFrame`
is, you guessed it, a `ttk.Frame`.

In [ ]:
class ControlFrame(ttk.Frame):
    """Full control of listbox. Set up as required.
    
    - Resize the listbox to correct size.
    - Place ARC buttons below the listbox in this way:
        
        [Add, Remove, Remove All]

    - Place the password entry (controlled by EncDecFrame)
    - Place the Enc and Dec buttons in this way:
    
        [Encrypt, Decrypt]
    """
    def __init__(self, *args, master, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        self._listbox = ListBox(*args, master=self, **kwargs)

        # pack with an internal padding otherwise the listbox will
        # look like shit.
        self._listbox.grid(
            row=0,
            column=0,
            ipadx=250,
            ipady=250,
            sticky='nsew',  # expand in all directions
        )

        ARCFrame(
            master=self,
            listbox=self._listbox,
        ).grid(
            row=1,
            column=0,
            sticky='nsew',  # expand in all directions
            ipady=5,
            pady=20,
        )
        EncDecFrame(
            master=self,
            listbox=self._listbox,
        ).grid(
            row=2,
            column=0,
            ipady=5,
            sticky='nsew',  # expand in all directions
        )

        self.columnconfigure(0, weight=1)
        self.rowconfigure(0, weight=4)  # the listbox must expand faster.
        self.rowconfigure(1, weight=1)
        self.rowconfigure(2, weight=1)

In [ ]:
root = tk.Tk()
cf = ControlFrame(master=root)

In [ ]:
cf.pack(expand=True, fill=tk.BOTH, padx=20, pady=20)
cf.mainloop()

In [ ]:
root.destroy()